# PART 1: THE SNOWPLOW PROBLEM

## Exercice 1.2

#### 1) Propose a n ∈ ℕ and a config of the houses

In [112]:
n = 5
config_difficile_1 = [-4.123, 10.4, 11.295, 12.05, 13.95]

- Pour le sort, la déneigeuse va d'abord aller à la première maison (pos: -4.123) puis aller aux cluster de maison (pos du cluster: [10, 14]), ce qui fait attendre le cluster de maisons plus longtemps qu'il ne le faudrait.

- Pour le greedy, la déneigeuse va également aller à la première maison (pos: -4.123) car c'est la plus proche. La déneigeuse aura donc fait attendre le cluster comme pour le sort.

#### 2. Proposer et implémenter un algorithme de complexité polynomiale.

In [113]:
# Import random
import random
from typing import List, Tuple

# Fonction make distance table
# Paramètres : liste de floats représentant la liste des adresses des maisons devant lesquelles passer dans l'ordre numérique croissant
# Retour : un tuple contenant une liste de floats représentant l'écart entre entre chaque étape du voyage (en prenant en compte le 0), et un int représentant l'incrément du 0
# La fonction a pour but de donner la liste des distances entre chaque étape du voyage
def make_distance_table(input_liste: List[float]) -> Tuple[List[float], int]:
    ret_list: List[float] = []
    zero_distance_inc: int = 0
    zero_added: bool = False
    for i in range(len(input_liste) - 1):
        # Mettre le 0 à 0 si l'adresse à 0 est positive
        if not zero_added and input_liste[i] > 0:
            zero_distance_inc = 0
            zero_added = True
        # Mettre le 0 à i + 1 si l'adresse à i est négative et l'adresse à i + 1 est positive
        if not zero_added and input_liste[i] < 0 and input_liste[i + 1] > 0:
            zero_distance_inc = i + 1
            zero_added = True
    # Mettre le 0 à la fin si l'adresse à la fin est négative
    if not zero_added:
        zero_distance_inc = len(input_liste)
        zero_added = True
    # Fabrication de la table des distances (distance des voyages entre chaque adresse adjacente)
    # Boucle pour les distances négatifs
    tmp_distance: int = 0
    for i in range(zero_distance_inc):
        ret_list.append(abs(input_liste[zero_distance_inc - (i + 1)]) + tmp_distance)
        tmp_distance = input_liste[zero_distance_inc - (i + 1)]
    # Reverse la liste des distances négatives (pour avoir les distances dans l'ordre croissant)
    ret_list.reverse()
    # Boucle pour les distances positifs
    tmp_pos_distance: List[float] = []
    tmp_distance = 0
    for i in range(len(input_liste) - zero_distance_inc):
        tmp_pos_distance.append(abs(input_liste[zero_distance_inc + i] - tmp_distance))
        tmp_distance = input_liste[zero_distance_inc + i]
    # Ajout des distances positifs à la liste de poids
    ret_list += tmp_pos_distance
    return (ret_list, zero_distance_inc)


# Fonction parcours
# Paramètres : liste de floats dans le désordre représentant la liste des adresses des maisons devant lesquelles passer
# Retour : liste de floats représentant la liste des adresses des maisons devant lesquelles passer dans l'ordre de passage
# La fonction a pour but de donner le parcours minimisant le temps d'attente moyen de chaque maison
def parcours(input_liste: List[float]) -> List[float]:
    # Trier la liste d'inout dans l'ordre croissant
    input_liste.sort()
    # On récupère la liste des distances entre chaque étape du voyage et l'incrément du 0 (position actuelle du snowplow)
    distance_table: List[float]
    distance_inc: int 
    distance_table, distance_inc = make_distance_table(input_liste)
    # On initialise le poids de chaque coté de la table de poids (nombre de maisons de chaque coté du snowplow)
    weight_negative: int = len(distance_table[:distance_inc])
    weight_positive: int = len(distance_table[distance_inc:])
    # On initialise la liste d'étapes du parcours
    parcours: List[float] = []
    # Boucle prncipale (chaque itération correspond à une étape du parcours)
    while len(input_liste) > 0:
        # Si il ne reste qu'une adresse, l'ajouter à la liste d'étapes et sortir de la boucle
        if len(input_liste) == 1:
            parcours.append(input_liste[0])
            break
        # Si le poids du coté négatif * le mouvement nécessaire coté positif est plus petit que le poids du coté positif * le mouvement nécessaire coté négatif
        if distance_inc < len(input_liste) and (weight_negative == 0  or distance_table[distance_inc - 1] * weight_positive > distance_table[distance_inc] * weight_negative):
            ## Opérations sur la liste d'addresse:
            # On ajoute l'adresse positive la plus proche comme prochaine étape
            parcours.append(input_liste[distance_inc])
            # On supprime cet élément de la liste d'adresses
            del input_liste[distance_inc]
            ## Opérations sur la distance des cotés:
            # Si le poids du coté négatif n'est pas déjà nul, l'augmenter de la valeur du poids de l'adresse positive la plus proche
            # if weight_negative != 0:
                # weight_negative += 1
            # Le poids du coté positif est réduit de la valeur du poids de l'adresse positive la plus proche
            weight_positive -= 1
            ## Opérations transfert de poids sur la table:
            # Si distance_inc est supérieur à 0, augmenter la valeur à distance_inc - 1 de la valeur du poids de l'adresse positive la plus proche
            if distance_inc > 0:
                distance_table[distance_inc - 1] += distance_table[distance_inc]
            # On supprime le poids de l'adresse positive la plus proche de la table de poids
            del distance_table[distance_inc]
        else:
            ## Opérations sur la liste d'addresse:
            # On ajoute l'adresse négative la plus proche comme prochaine étape
            parcours.append(input_liste[distance_inc - 1])
            # On supprime cet élément de la liste d'adresses
            del input_liste[distance_inc - 1]
            ## Opérations sur le poids des cotés:
            # Le poids du coté négatif est réduit de la valeur du poids de l'adresse négative la plus proche
            weight_negative -= 1
            # Si le poids du coté positif n'est pas déjà nul, l'augmenter de la valeur du poids de l'adresse négative la plus proche
            # if weight_positive != 0:
                # weight_positive += 1
            ## Opérations transfert de poids sur la table:
            # Si distance_inc est inférieur à la taille de la table de poids, augmenter la valeur à distance_inc de la valeur du poids de l'adresse négative la plus proche
            if distance_inc < len(distance_table):
                distance_table[distance_inc] += distance_table[distance_inc - 1]
            # On supprime le poids de l'adresse négative la plus proche de la table de poids
            del distance_table[distance_inc - 1]
            # On décrémente distance_inc
            distance_inc -= 1
    return parcours
    

#### Tests

In [114]:
def greedy(input_liste: List[float]) -> List[float]:
    # Sort list
    input_liste.sort()
    # Parcours
    parcours: List[float] = []
    # Position actuelle du snowplow
    snowplow_position: float = 0
    # Boucle principale
    # A chaque étape, on va chercher l'adresse la plus proche de la position actuelle du snowplow
    while len(input_liste) > 0:
        # On cherche l'adresse la plus proche de la position actuelle du snowplow
        closest_address: float = input_liste[0]
        closest_address_index: int = 0
        for i in range(len(input_liste)):
            if abs(input_liste[i] - snowplow_position) < abs(closest_address - snowplow_position):
                closest_address = input_liste[i]
                closest_address_index = i
        # On ajoute l'adresse la plus proche à la liste d'étapes
        parcours.append(closest_address)
        # On supprime l'adresse la plus proche de la liste d'adresses
        del input_liste[closest_address_index]
        # On met à jour la position actuelle du snowplow
        snowplow_position = closest_address
    return parcours
        


def calcul_average_wait_time(input_liste: List[float]) -> float:
    # Actual wait time
    actual_wait_time: float = 0
    # Total des temps d'attente
    total_wait_time: float = 0
    # Precedente adresse
    previous_address: float = 0
    # Boucle pour chaque adresse
    for i in range(len(input_liste)):
        # Ajouter le temps d'attente à total_wait_time
        if previous_address < input_liste[i]:
            actual_wait_time += input_liste[i] - previous_address
        elif previous_address > input_liste[i]:
            actual_wait_time += previous_address - input_liste[i]
        # Ajouter le temps d'attente à total_wait_time
        total_wait_time += actual_wait_time
        # Mettre l'adresse actuelle dans previous_address
        previous_address = input_liste[i]
    # Retourner le temps d'attente moyen
    return total_wait_time / len(input_liste)


# Test make_distance_table avec [-4, -3, -2, -1, 1, 2, 3, 4], devrait retourner ([1, 1, 1, 1, 1, 1, 1, 1], 4)
if make_distance_table([-4, -3, -2, -1, 1, 2, 3, 4]) != ([1, 1, 1, 1, 1, 1, 1, 1], 4):
    print("Erreur dans make_distance_table, devrait retourner ([1, 1, 1, 1, 1, 1, 1, 1], 4) mais retourne", make_distance_table([-4, -3, -2, -1, 1, 2, 3, 4]))
# Test make_distance_table avec [-8, -3, -2, -1, 1, 2, 3], devrait retourner ([5, 3, 2, 1, 1, 2, 3], 4)
if make_distance_table([-8, -3, -2, -1, 1, 2, 3]) != ([5, 1, 1, 1, 1, 1, 1], 4):
    print("Erreur dans make_distance_table, devrait retourner ([5, 1, 1, 1, 1, 1, 1], 4) mais retourne", make_distance_table([-8, -3, -2, -1, 1, 2, 3]))
# Test make_distance_table avec [-8, -3, -2, -1, 1, 2, 3, 8], devrait retourner ([5, 1, 1, 1, 1, 1, 1, 5], 4)
if make_distance_table([-8, -3, -2, -1, 1, 2, 3, 8]) != ([5, 1, 1, 1, 1, 1, 1, 5], 4):
    print("Erreur dans make_distance_table, devrait retourner ([5, 1, 1, 1, 1, 1, 1, 5], 4) mais retourne", make_distance_table([-8, -3, -2, -1, 1, 2, 3, 8]))
    
print()    

# Test temps d'attentes moyens
# Test avec [1, -5, -10, 5]
print("Temps d'attente moyen pour [1, -5, -10, 5]:", calcul_average_wait_time([1, -5, -10, 5]))
# Test avec [1, 5, -5, -10]
print("Temps d'attente moyen pour [1, 5, -5, -10]:", calcul_average_wait_time([1, 5, -5, -10]))
# Test avec [1, -5, -10, 5, 10]
print("Temps d'attente moyen pour [10, -5, 1, 5, -10]:", calcul_average_wait_time([1, -5, -10, 5, 10]))
# Test avec [1, 5, 10, -5, -10]
print("Temps d'attente moyen pour [1, 5, 10, -5, -10]:", calcul_average_wait_time([1, 5, 10, -5, -10]))

print()

# TTest basique 1, parcours avec [-10, 1], devrait retourner [1, -10]
if parcours([-10, 1]) != [1, -10]:
    print("Test basiquue 1: devrait retourner [-1, 10] mais retourne", parcours([-10, 1]))
else:
    print("Test basique 1: bon parcours obtenu: [1, -10], temps d'attente moyen:", calcul_average_wait_time(parcours([-10, 1])))
# Test basique 2, parcours avec [-10, -5, 1], devrait retourner [1, -5, -10]
if parcours([-10, -5, 1]) != [1, -5, -10]:
    print("Test basiquue 2: devrait retourner [1, -5, -10] mais retourne", parcours([-10, -5, 1]))
else:
    print("Test basique 2: bon parcours obtenu: [1, -5, -10], temps d'attente moyen:", calcul_average_wait_time(parcours([-10, -5, 1])))
    
print()

# Test non sort 1, parcours avec [-10, -5, 1, 5], devrait retourner [1, 5, -5, -10]
if parcours([-10, -5, 1, 5]) != [1, 5, -5, -10]:
    print("Test non sort 1: devrait retourner [1, 5, -5, -10] mais retourne", parcours([-10, -5, 1, 5]))
else:
    print("Test non sort 1: bon parcours obtenu: [1, 5, -5, -10], temps d'attente moyen:", calcul_average_wait_time(parcours([-10, -5, 1, 5])))
# Test non sort 2, parcours avec [10, -5, 1, 5, -10], devrait retourner [1, -5, -10, 5, 10]
if parcours([10, -5, 1, 5, -10]) != [1, 5, 10, -5, -10]:
    print("Test non sort 2: devrait retourner [1, 5, 10, -5, -10] mais retourne", parcours([10, -5, 1, 5, -10]))
else:
    print("Test non sort 2: bon parcours obtenu: [1, 5, 10, -5, -10], temps d'attente moyen:", calcul_average_wait_time(parcours([10, -5, 1, 5, -10])))
    

print()    

# Test only negative 1, parcours avec [-10, -10, -10, -10, -10], devrait retourner [-10, -10, -10, -10, -10]
if parcours([-10, -10, -10, -10, -10]) != [-10, -10, -10, -10, -10]:
    print("Test only negative 1: devrait retourner [-10, -10, -10, -10, -10] mais retourne", parcours([-10, -10, -10, -10, -10]))
else:
    print("Test only negative 1: bon parcours obtenu: [-10, -10, -10, -10, -10], temps d'attente moyen:", calcul_average_wait_time(parcours([-10, -10, -10, -10, -10])))
# Tes only positive 1, parcours avec [10, 10, 10, 10, 10], devrait retourner [10, 10, 10, 10, 10]
if parcours([10, 10, 10, 10, 10]) != [10, 10, 10, 10, 10]:
    print("Test only positive 1: devrait retourner [10, 10, 10, 10, 10] mais retourne", parcours([10, 10, 10, 10, 10]))
else:
    print("Test only positive 1: bon parcours obtenu: [10, 10, 10, 10, 10], temps d'attente moyen:", calcul_average_wait_time(parcours([10, 10, 10, 10, 10])))
    
print()

# Test edge 1, parcours avec [-10, -10, -10, -10, -10, -10, -10, -10, -10, 1] devrait retourner [1, -10, -10, -10, -10, -10, -10, -10, -10, -10]
if parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1]) != [1, -10, -10, -10, -10, -10, -10, -10, -10, -10]:
    print("Test edge 1: devrait retourner [1, -10, -10, -10, -10, -10, -10, -10, -10, -10] mais retourne", parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1]))
else:
    print("Test edge 1: bon parcours obtenu: [1, -10, -10, -10, -10, -10, -10, -10, -10, -10], temps d'attente moyen:", calcul_average_wait_time(parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1])))
# Test edge 2, parcours avec [-10, -10, -10, -10, -10, -10, -10, -10, -10, 1, 10] devrait retourner [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10]
if parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1, 10]) != [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10]:
    print("Test edge 2: devrait retourner [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10] mais retourne", parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1, 10]))
else:
    print("Test edge 2: bon parcours obtenu: [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10], temps d'attente moyen:", calcul_average_wait_time(parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1, 10])))
# Test edge 3, parcours avec [-10, -10, -10, -10, -10, -10, -10, -10, -10, 1, 10, 10] devrait retourner [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10, 10]
if parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1, 10, 10]) != [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10, 10]:
    print("Test edge 3: devrait retourner [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10, 10] mais retourne", parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1, 10, 10]))
else:
    print("Test edge 3: bon parcours obtenu: [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10, 10], temps d'attente moyen:", calcul_average_wait_time(parcours([-10, -10, -10, -10, -10, -10, -10, -10, -10, 1, 10, 10])))
# Test edge 4, parcours avec [-1000, -4, 1, 10, 1000000] devrait retourner [1, -4, 10, -1000, 1000000]
if parcours([-1000, -4, 1, 10, 1000000]) != [1, -4, 10, -1000, 1000000]:
    print("Test edge 4: devrait retourner [1, -4, 10, -1000, 1000000] mais retourne", parcours([-1000, -4, 1, 10, 1000000]))
else:
    print("Test edge 4: bon parcours obtenu: [1, -4, 10, -1000, 1000000], temps d'attente moyen:", calcul_average_wait_time(parcours([-1000, -4, 1, 10, 1000000])))
    
print()

# Test random 1
if parcours([-4.123, 10.4, 11.295, 12.05, 13.95]) != [10.4, 11.295, 12.05, 13.95, -4.123]:
    print("Test random 1: devrait retourner [10.4, 11.295, 12.05, 13.95, -4.123] mais retourne", parcours([-4.123, 10.4, 11.295, 12.05, 13.95]))
else:
    print("Test random 1: bon parcours obtenu: [10.4, 11.295, 12.05, 13.95, -4.123], temps d'attente moyen:", calcul_average_wait_time(parcours([-4.123, 10.4, 11.295, 12.05, 13.95])))
    
print()
    
# Test spécifique
# test = [-89.38051710594593, -32.628255285402545, -13.462371625667771, -3.6264607831936644, 14.503083059151066]
# print("Teste spécifique: " + str(test))
# test = parcours(test)
# print("Résultat:" + str(test))

print()
    
# Generer une liste random de float de taille comprise entre 1 et 20, dans l'ordre croissant
liste_random: List[float] = []
for i in range(random.randint(1, 20)):
    liste_random.append(random.uniform(-100, 100))
liste_random.sort()
print("Liste random: " + str(liste_random))
# Tester la fonction parcours sur la liste random
parcours_random: List[float] = parcours(liste_random)
print("Parcours random: " + str(parcours_random))
print("Temp moyen: " + str(calcul_average_wait_time(parcours_random)))

print()

import numpy

# Test de ratio de temps avec une liste de 1000 éléments
# Génération d'une liste random de 1000 éléments compris entre -1000 et 1000
print("Test de ratio de temps avec une liste de 1000 éléments")
random_list: List[float] = numpy.random.normal(0,1000,1000).tolist()
# Calcul du temps d'attente moyen pour le parcours greedy
parcours_greedy: List[float] = greedy(random_list.copy())
# Calcul du temps d'attente moyen pour le parcours parcours
parcours_parcours: List[float] = parcours(random_list.copy())
# Ratio de temps
print("Ratio de temps: " + str(calcul_average_wait_time(parcours_parcours) / calcul_average_wait_time(parcours_greedy)))


Temps d'attente moyen pour [1, -5, -10, 5]: 11.75
Temps d'attente moyen pour [1, 5, -5, -10]: 10.25
Temps d'attente moyen pour [10, -5, 1, 5, -10]: 15.8
Temps d'attente moyen pour [1, 5, 10, -5, -10]: 14.2

Test basique 1: bon parcours obtenu: [1, -10], temps d'attente moyen: 6.5
Test basique 2: bon parcours obtenu: [1, -5, -10], temps d'attente moyen: 6.666666666666667

Test non sort 1: devrait retourner [1, 5, -5, -10] mais retourne [1, -5, -10, 5]
Test non sort 2: devrait retourner [1, 5, 10, -5, -10] mais retourne [1, 5, -5, -10, 10]

Test only negative 1: bon parcours obtenu: [-10, -10, -10, -10, -10], temps d'attente moyen: 10.0
Test only positive 1: bon parcours obtenu: [10, 10, 10, 10, 10], temps d'attente moyen: 10.0

Test edge 1: bon parcours obtenu: [1, -10, -10, -10, -10, -10, -10, -10, -10, -10], temps d'attente moyen: 10.9
Test edge 2: bon parcours obtenu: [1, -10, -10, -10, -10, -10, -10, -10, -10, -10, 10], temps d'attente moyen: 12.818181818181818
Test edge 3: bon par